# que-A cloth manufacturing company is interested to know about the segment or attributes causes high sale. Approach - A random forest is built with target variable Sale (we will first convert it in categorical variable) & all other variable will be independent in the analysis.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets  
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import  DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import  DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('Company_Data.csv')
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [4]:
label_encoder = preprocessing.LabelEncoder()
df['ShelveLoc']= label_encoder.fit_transform(df['ShelveLoc']) 
df['Urban']=label_encoder.fit_transform(df['Urban'])
df['US']=label_encoder.fit_transform(df['US'])

In [5]:
df.insert(11,'Sales_avg'," ")

for i in range(0,len(df['Sales']),1):
    if df['Sales'].iloc[i]<=7.5:
        df['Sales_avg'].iloc[i]='Low'
    else:
        df['Sales_avg'].iloc[i]='high'

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [6]:
df.drop('Sales',axis=1,inplace=True)

In [7]:
x=df.iloc[:,0:-1]
y=df['Sales_avg']

In [8]:
X_train, X_test,y_train,y_test = train_test_split(x,y, test_size=0.2,random_state=40)

In [9]:
model = DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [10]:
model.score(X_test,y_test)

0.5875

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [12]:
rfc=RandomForestClassifier(random_state=42)

# Using Grid Search

In [13]:
param_grid = { 
    'n_estimators': [100,150,200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,4,10,12,15],
    'criterion' :['gini', 'entropy']
}

In [16]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 10, 12, 15],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 150, 200]})

In [17]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 150}

In [18]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 150,
                            max_depth=4, criterion='gini')

In [19]:
rfc1.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, n_estimators=150, random_state=42)

In [20]:

pred=rfc1.predict(X_test)

In [21]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.7875


# Using Random search

In [22]:
import numpy as np 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(151, 1200, 10, dtype = int))}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 80, 
                               cv = 4, verbose= 5, random_state= 101, n_jobs = -1)
model.fit(X_train,y_train)

Fitting 4 folds for each of 80 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  1.7min finished


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_iter=80,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 142, 274, 406, 538,
                                                      671, 803, 935, 1067, 1200,
                                                      None],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [4, 6, 8, 12],
                                        'min_samples_split': [5, 7, 10, 14],
                                        'n_estimators': [151, 267, 384, 500,
                                                         617, 733, 850, 966,
                                                         1083, 1200]},
                   random_state=101, verbose=5)

In [23]:
model.best_params_

{'n_estimators': 267,
 'min_samples_split': 5,
 'min_samples_leaf': 6,
 'max_features': 'sqrt',
 'max_depth': 671,
 'criterion': 'entropy'}

In [24]:
rfc2=RandomForestClassifier(random_state=42,n_estimators= 733,min_samples_split= 7,min_samples_leaf= 8,max_depth=1067, criterion='entropy')

In [25]:
rfc2.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=1067, min_samples_leaf=8,
                       min_samples_split=7, n_estimators=733, random_state=42)

In [26]:
pred=rfc1.predict(X_test)

In [27]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.7875


# Classification report and confusion matrix

In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [29]:
pre = model.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
acc3 = accuracy_score(y_test,pred)

[[35 10]
 [ 7 28]]
              precision    recall  f1-score   support

         Low       0.83      0.78      0.80        45
        high       0.74      0.80      0.77        35

    accuracy                           0.79        80
   macro avg       0.79      0.79      0.79        80
weighted avg       0.79      0.79      0.79        80



In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pre)

0.75

In [31]:
from sklearn import ensemble

# Creating classifier Object
ada = ensemble.AdaBoostClassifier()
#Fitting the classifier to training data
ada.fit(X_train,y_train)

# Making Predictions
ada_pred = ada.predict(X_test)

print("Traing Score:%f"%ada.score(X_train,y_train))
print("Testing Score:%f"%ada.score(X_test,y_test))

Traing Score:0.950000
Testing Score:0.800000
